# Paraphrase Generator with Adversarial Discriminator

## 1. ETL

In [1]:
import pickle
with open("./data/dictionary.pkl", "rb") as file:
    dictionary = pickle.load(file)

with open("./data/adversary_100.pkl", "rb") as file:
    nmt_dataset = pickle.load(file)

# with open("./Datasets/qq_dataset.pkl", "rb") as file:
#     qq_dataset = pickle.load(file)

## 2. EDA

## 3. Preprocessing

## 4. Preparing the dataloader

In [2]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
import torch

pad_idx = dictionary.word2idx['<pad>'] ##get the pad index from the vocab

def collate_batch(batch):
    sent_list, synt_lst, trg_list = [], [], []
    # print(len(batch))
    # sens_, syns_, trgs_ = batch
    # for sen_, syn_, trg_ in zip(sens_, syns_, trgs_):
    for sen_, syn_, trg_, adv_ in batch:
        processed_sent = torch.tensor(sen_, dtype=torch.int64)
        sent_list.append(processed_sent)
        processed_synt = torch.tensor(syn_, dtype=torch.int64)
        synt_lst.append(processed_synt)
        processed_trg = torch.tensor(trg_, dtype=torch.int64)
        trg_list.append(processed_trg)
        adv_ = torch.tensor(adv_, dtype=torch.int64)
        
    return pad_sequence(sent_list, padding_value=pad_idx, batch_first=True), pad_sequence(synt_lst, padding_value=pad_idx, batch_first=True), pad_sequence(trg_list, padding_value=pad_idx, batch_first=True), adv_

In [3]:
from torch.utils.data import DataLoader
import random

random.seed(6969)
random.shuffle(nmt_dataset)

train_range = int(len(nmt_dataset) * 0.7)

train_set = nmt_dataset[:train_range]
val_set   = nmt_dataset[train_range:]
# test_set = train_data[90:]

train_dataloader = DataLoader(train_set, batch_size=32, shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(val_set, batch_size=32, shuffle=True, collate_fn=collate_batch)

In [4]:
for idx, (sen, syn, trg, adv) in enumerate(train_dataloader):
    print(sen.shape)
    print(syn.shape)
    print(trg.shape)
    print(adv.shape)
    break

torch.Size([32, 60])
torch.Size([32, 174])
torch.Size([32, 62])
torch.Size([74])


## 5. model

In [5]:
import torch, torchdata, torchtext
from torch import nn
import torch.nn.functional as F
from tqdm import tqdm
import random, math, time
from torch.autograd import Variable
import operator
import numpy as np

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

#make our work comparable if restarted the kernel
SEED = 6969
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

cpu


## Positional Encoding
$
PE_{pos,2i}     =   sin(\frac{pos}{10000^\frac{2i}{d_{model}}})\\
PE_{pos,2i+1}   =   cos(\frac{pos}{10000^\frac{2i}{d_{model}}})
$

In [6]:
class PositionalEncoding(nn.Module):
    def __init__(self, dim_model, dropout = 0.1, max_len = 5000):
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        
        # Encoding - From formula
        pos_encoding = torch.zeros(max_len, dim_model)
        positions_list = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        division_term = torch.exp(torch.arange(0, dim_model, 2).float() * (-np.log(10000.0)) / dim_model) # 1000^(2i/dim_model)
        
        # PE(pos, 2i) = sin(pos/1000^(2i/dim_model))
        pos_encoding[:, 0::2] = torch.sin(positions_list * division_term)
        
        # PE(pos, 2i + 1) = cos(pos/1000^(2i/dim_model))
        pos_encoding[:, 1::2] = torch.cos(positions_list * division_term)
        
        # Saving buffer (same as parameter without gradients needed)
        pos_encoding = pos_encoding.unsqueeze(0).transpose(0, 1)
        self.register_buffer("pos_encoding",pos_encoding)
        
    def forward(self, token_embedding):
        # Residual connection + pos encoding
        token_embedding = token_embedding + self.pos_encoding[:token_embedding.size(0), :]
        return self.dropout(token_embedding)

In [7]:
class Discriminator(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.sent_layernorm_embedding = nn.LayerNorm(emb_dim, elementwise_affine=False)
        self.adv = nn.Linear(emb_dim, 74)
        
    def forward(self, sent_embeddings):
        x = self.sent_layernorm_embedding(sent_embeddings).squeeze(1)
        print('Layer Norm',x.shape)
        x = self.adv(x)
        print('Linear Layer',x.shape)
        return x

In [8]:
class Transformer_Adversary(nn.Module):
    def __init__(self, input_dim, emb_dim, device, word_dropout = 0.4, dropout = 0.1):
        super(Transformer_Adversary, self).__init__()
        self.input_dim = input_dim 
        self.emb_dim = emb_dim 
        self.word_dropout = word_dropout
        self.dropout = dropout
        self.device = device
        
        self.scale = np.sqrt(self.emb_dim)
        # self.scale = torch.sqrt(torch.IntTensor([self.hid_dim])).to(device)
        # synt
        # vocabulary embedding
        self.embedding_encoder = nn.Embedding(input_dim, emb_dim)
        self.embedding_decoder = nn.Embedding(input_dim, emb_dim)
        # positional encoding
        self.pos_encoder = PositionalEncoding(emb_dim, dropout = 0.0)

        self.transformer = nn.Transformer(d_model = emb_dim, nhead = 12, dropout = dropout)
        # linear Transformation
        self.linear = nn.Linear(emb_dim, input_dim)

        self.init_weights()

        self.adversary = Discriminator(emb_dim)
        
    def load_embedding(self, embedding):  #synPG applied with GloVe glove.840B.300d.txt
        self.embedding_encoder.weight.data.copy_(torch.from_numpy(embedding)) 
        self.embedding_decoder.weight.data.copy_(torch.from_numpy(embedding))  

    def init_weights(self):
        initrange = 0.1
        # initialize cocabulary matrix weight
        self.embedding_encoder.weight.data.uniform_(-initrange, initrange)
        self.embedding_decoder.weight.data.uniform_(-initrange, initrange)
        # initialize linear weight
        self.linear.weight.data.uniform_(-initrange, initrange)
        self.linear.bias.data.fill_(0)

    def generate_square_mask(self, max_sent_len, max_synt_len):
        size = max_sent_len + max_synt_len + 2 #<sos> and <eos>
        mask = torch.zeros((size, size))
        mask[:max_sent_len, max_sent_len:] = float("-inf")
        mask[max_sent_len:, :max_sent_len] = float("-inf")
        return mask
        
    def forward(self, sents, synts, trg):
        #sents  : batch_size, seq_len
        #synts  : batch_size, seq_len
        #trgs   : batch_size, seq_len 
        batch_size   = sents.size(0)
        max_sent_len = sents.size(1)
        max_synt_len = synts.size(1) - 2    # count without <sos> and <eos>
        max_targ_len = trg.size(1) - 2      # count without <sos> and <eos>

        # apply word dropout
        drop_mask = torch.bernoulli(self.word_dropout * torch.ones(max_sent_len)).bool().to(self.device)
        sents = sents.masked_fill(drop_mask, -1e10)

        # sentence, syntax => embedding
        sent_embeddings = self.embedding_encoder(sents).transpose(0, 1) * self.scale
        #sent_emb = [seq_len, batch size, emb_size]
        synt_embeddings = self.embedding_encoder(synts).transpose(0, 1) * self.scale
        synt_embeddings = self.pos_encoder(synt_embeddings) 
        #synt_emb = [seq_len, batch size, emb_size]
        en_embeddings = torch.cat((sent_embeddings, synt_embeddings), dim=0)
        #en_emb = [seq_len, batch size, emb_size*2]

        # do not allow cross attetion
        src_mask = self.generate_square_mask(max_sent_len, max_synt_len).to(self.device)
        
        # target => embedding
        de_embeddings = self.embedding_decoder(trg[:, :-1]).transpose(0, 1) * self.scale
        de_embeddings = self.pos_encoder(de_embeddings)
        
        # sequential mask
        trg_mask = self.transformer.generate_square_subsequent_mask(max_targ_len+1).to(self.device)
        
        # forward
        outputs = self.transformer(en_embeddings, de_embeddings, src_mask=src_mask, tgt_mask=trg_mask)
        
        encoder_outputs = outputs.encoder(sent_embeddings)

        # apply linear layer to vocabulary size
        outputs = outputs.transpose(0, 1)
        outputs = self.linear(outputs.contiguous().view(-1, self.emb_dim))
        outputs = outputs.view(batch_size, max_targ_len + 1, self.input_dim)
        #output = [batch size, trg_len, vocab_size]

        # discriminator
        for p in self.adversary.parameters():
            p.required_grad=False

        adv_outputs = self.adversary(sent_embeddings)        

        return outputs, adv_outputs
    
    def forward_token(self, sents):
        drop_mask = torch.bernoulli(self.word_dropout*torch.ones(sents.shape)).bool().to(self.device)
        sents = sents.masked_fill(drop_mask, -1e10)
               
        input_token_embeds = self.embedding_encoder(sents) + self.pos_encoder(sents)
        x = self.layernorm_embedding(input_token_embeds)
        
        x = F.dropout(x, p=self.dropout)
        
        x = x.transpose(0, 1)

        for encoder_layer in self.transformer.encoder_layer:
            x = encoder_layer(x)
            
        x = x.transpose(0, 1)
        return x

    def forward_adv(self, sents):
        for p in self.adversary.parameters():
            p.required_grad=True
        
        # sent_embeddings = self.embedding_encoder(sents).transpose(0, 1).detach()
        sent_embeddings = self.forward_token(sents).detach()
        print('Sentence Embedding', sent_embeddings.shape)

        adv_outputs = self.adversary(sent_embeddings)
        print('Adversary', adv_outputs.shape)
        return adv_outputs

    def generate(self, sents, synts, max_len = 30, sample=True, temp=0.5):
        #sents  : batch_size, seq_len
        #synts  : batch_size, seq_len
        batch_size   = sents.size(0)
        max_sent_len = sents.size(1)
        max_synt_len = synts.size(1) - 2  # count without <sos> and <eos>
        max_targ_len = max_len
        
        # output index starts with <sos>
        idxs = torch.zeros((batch_size, max_targ_len+2), dtype=torch.long).to(self.device)
        idxs[:, 0] = 1
        
        # sentence, syntax => embedding
        sent_embeddings = self.embedding_encoder(sents).transpose(0, 1) * self.scale
        synt_embeddings = self.embedding_encoder(synts).transpose(0, 1) * self.scale
        synt_embeddings = self.pos_encoder(synt_embeddings)
        en_embeddings = torch.cat((sent_embeddings, synt_embeddings), dim=0)
        
        # do not allow cross attetion
        src_mask = self.generate_square_mask(max_sent_len, max_synt_len).to(self.device)
        
        # starting index => embedding
        de_embeddings = self.embedding_decoder(idxs[:, :1]).transpose(0, 1) * self.scale
        de_embeddings = self.pos_encoder(de_embeddings)
        
        # sequential mask
        trg_mask = self.transformer.generate_square_subsequent_mask(de_embeddings.size(0)).to(self.device)
        
        # encode
        memory = self.transformer.encoder(en_embeddings, mask=src_mask)
        
        # auto-regressively generate output
        for i in range(1, max_targ_len+2):
            if i % 5 == 0:
                print(f'epoch : {i}')
            # decode
            outputs = self.transformer.decoder(de_embeddings, memory, tgt_mask=trg_mask)
            outputs = self.linear(outputs[-1].contiguous().view(-1, self.emb_dim))
            
            # get argmax index or sample index
            if not sample:
                values, idx = torch.max(outputs, 1)
            else:
                probs = F.softmax(outputs/temp, dim=1)
                idx = torch.multinomial(probs, 1).squeeze(1)
            
            # save to output index
            idxs[:, i] = idx
            
            # concatenate index to decoding
            de_embeddings = self.embedding_decoder(idxs[:, :i+1]).transpose(0, 1) * self.scale
            de_embeddings = self.pos_encoder(de_embeddings)
            
            # new sequential mask
            trg_mask = self.transformer.generate_square_subsequent_mask(de_embeddings.size(0)).to(self.device)
        
        return idxs[:, 1:]

### FastText Embedding

In [9]:
# from torchtext.vocab import FastText
# fast_vectors = FastText(language='simple') ##Load fasttext with language=simple
# fast_embedding = fast_vectors.get_vecs_by_tokens(vocab_transform.get_itos()).to(device)

### Glove Embedding

In [10]:
from gensim.test.utils import datapath

#you have to put this file in some python/gensim directory; just run it and it will inform where to put....
glove_file = datapath('glove.6B.300d.txt')

In [11]:
with open("./data/dictionary.pkl", "rb") as file:
    dictionary = pickle.load(file)

vocab_dict = dictionary.word2idx

In [12]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

from utils import load_embedding

input_dim = len(vocab_dict)
emb_dim = 300  #fasttext
word_dropout = 0.4 #following SynPG
dropout     = 0.1

embedding = load_embedding(glove_file, dictionary)

model = Transformer_Adversary(input_dim=input_dim, emb_dim = emb_dim, device=device, word_dropout = word_dropout, dropout = dropout)
model = model.to(device)
model.load_embedding(embedding)
# model.embedding_encoder.weight.data = fast_embedding #apply fasttext instead of Glove 840b 300d.txt (5.56 GB) TT
# model.embedding_decoder.weight.data = fast_embedding

cpu
load 22696 of 31414 from pretrained word embeddings



In [13]:
#check_model
# sents   = torch.LongTensor(16, 30).random_(0, 10) #sents: [batch size, seq_len]
# synts   = torch.LongTensor(16, 30).random_(0, 10) #synts: [batch size, seq_len]
# output  = model.generate(sents, synts)
# output.shape

## 6. Training

In [14]:
from tqdm import tqdm

def train(model, loader, epoch, para_optimizer, adv_optimizer, para_criterion, adv_criterion, clip, loader_length, accumulation_steps = 1):
    
    model.train()

    for idx, (sents_, synts_, trgs_, adv_targs) in tqdm(enumerate(loader)):

        epoch_loss = 0
        adv_total_loss = 0.0	   

        batch_size   = sents_.size(0)
        max_sent_len = sents_.size(1)
        max_synt_len = synts_.size(1) - 2  # count without <sos> and <eos>
        
        # Put input into device
        sents_ = sents_.to(device)
        synts_ = synts_.to(device)
        trgs_ = trgs_.to(device)
        adv_targs = adv_targs.to(device)

        #optimize adv
        outputs = model.forward_adv(sents_)
        # print(outputs)
        # print(outputs.shape)
        
        loss = adv_criterion(outputs, adv_targs)
        loss.backward()
        adv_total_loss += loss.item()

        if (idx+1) % accumulation_steps == 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
            if epoch > 1:
                adv_criterion.step()
        adv_optimizer.zero_grad()

        #optimize model
        outputs, adv_outputs = model(sents_, synts_, trgs_)
        # calculate loss
        targs_ = trgs_[:, 1:].contiguous().view(-1) #Without <SOS>
        outputs_ = outputs.contiguous().view(-1, outputs.size(-1))
        loss = para_criterion(outputs_, targs_)

        if epoch > 1:
            loss -= 0.1 * adv_criterion(adv_outputs, adv_targs)
        loss.backward()
        epoch_loss += loss.item()

        if (idx+1) % accumulation_steps == 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
            para_optimizer.step()
            para_optimizer.zero_grad()

    return epoch_loss / loader_length

def evaluate(model, loader, para_criterion, adv_criterion, loader_length):

    #turn off dropout (and batch norm if used)
    model.eval()
    para_loss = 0
    adv_loss = 0
    with torch.no_grad():

        for sents_, synts_, trgs_, adv_targs in tqdm(loader):

            batch_size   = sents_.size(0)
            max_sent_len = sents_.size(1)
            max_synt_len = synts_.size(1) - 2  # count without <sos> and <eos>
            
            # Put input into device
            sents_ = sents_.to(device)
            synts_ = synts_.to(device)
            trgs_ = trgs_.to(device)
            adv_targs = adv_targs.to(device)

            #forward 
            outputs, adv_outputs = model(sents_, synts_, trgs_)
            targs_ = trgs_[:, 1:].contiguous().view(-1) #Without <SOS>
            outputs_ = outputs.contiguous().view(-1, outputs.size(-1))

            para_loss += para_criterion(outputs, targs_) 
            adv_loss += adv_criterion(adv_outputs, adv_targs)
            
        
    return para_loss / loader_length, adv_loss / loader_length

In [15]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [16]:
import torch.optim as optim

pad_idx = dictionary.word2idx['<pad>'] ##get the pad index from the vocab

lr = 10e-4 #Following SynPG
wd = 10e-5 #Following SynPG
#training hyperparameters
para_optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
adv_optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=0)

para_criterion = nn.CrossEntropyLoss(ignore_index=pad_idx).to(device)
adv_criterion = nn.BCEWithLogitsLoss().to(device)

In [17]:
train_loader_length = len(list(iter(train_dataloader)))
val_loader_length   = len(list(iter(valid_dataloader)))

In [18]:
import time
import math
best_valid_loss = float('inf')
num_epochs = 5
clip       = 1

save_path = f'models/adversary_nmt.pt' #Change here

train_losses = []
valid_losses = []

for epoch in range(num_epochs):

    start_time = time.time()

    # training 
    train_loss = train(model, train_dataloader, epoch, para_optimizer, adv_optimizer, para_criterion, adv_criterion, clip, train_loader_length)
    para_loss, adv_loss = evaluate(model, valid_dataloader, epoch, para_optimizer, adv_optimizer, para_criterion, adv_criterion, clip, val_loader_length)
    
    valid_loss = para_loss - 0.1 * adv_loss

    #for plotting
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    # save model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), save_path)
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')
    
    #lower perplexity is better

0it [00:00, ?it/s]


IndexError: index out of range in self

## Plot the losses

In [ ]:
#code here
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(5, 3))
ax = fig.add_subplot(1, 1, 1)
ax.plot(train_losses, label = 'train loss')
ax.plot(valid_losses, label = 'valid loss')
plt.legend()
ax.set_xlabel('updates')
ax.set_ylabel('loss')

## 7. Inference

In [ ]:
# for sents_, synts_, trgs_ in valid_dataloader:
#     break

# sents_ = sents_[0:4]
# synts_ = synts_[0:4]
# trgs_  = trgs_[0:4]

# outputs = model(sents_, synts_, trgs_)

In [ ]:
from utils import synt2str, sent2str, load_embedding, reverse_bpe
    
def generate(model, loader, criterion, loader_length, vocab_transform):
    #turn off dropout (and batch norm if used)
    model.eval()
    epoch_loss = 0
    with open("./eval/target_sents.txt", "w") as fp1, \
         open("./eval/target_synts.txt", "w") as fp2, \
         open("./eval/outputs.txt", "w") as fp3:
        with torch.no_grad():
            for sents_, synts_, trgs_ in loader:

                batch_size   = sents_.size(0)
                max_sent_len = sents_.size(1)
                max_synt_len = synts_.size(1) - 2  # count without <sos> and <eos>
                
                # generate
                idxs = model.generate(sents_, synts_, sents_.size(1), temp=0.5)
                
                # write output
                for sent, idx, targ, synt_ in zip(sents_, idxs.cpu().numpy(), trgs_, synts_):
                    fp1.write(targ+'\n')
                    fp2.write(synt_+'\n')
                    fp3.write(reverse_bpe(synt2str(idx, vocab_transform))+'\n')

In [ ]:
model.load_state_dict(torch.load(save_path))
generate(model, valid_dataloader, criterion, val_loader_length, dictionary)

## 8. Metric

In [ ]:
import numpy as np
from nltk.translate.bleu_score import sentence_bleu

def cal_bleu(hypothesis, reference, n):
    hypothesis = hypothesis.strip().split(' ')
    reference = reference.strip().split(' ')

    if n == 0:
        return sentence_bleu([reference], hypothesis)
    elif n == 1:
        weights = (1, 0, 0, 0)
    elif n == 2:
        weights = (0, 1, 0, 0)
    elif n == 3:
        weights = (0, 0, 1, 0)
    elif n == 4:
        weights = (0, 0, 0, 1)

    return sentence_bleu([reference], hypothesis, weights=weights)     

In [ ]:
with open('./eval/target_sents.txt') as fp:
    targs = fp.readlines()
with open('./eval/outputs.txt') as fp:
    preds = fp.readlines()

print(f"number of examples: {len(preds)}")

scores = [cal_bleu(pred, targ, 0) for pred, targ in zip(preds, targs)]

print(f"BLEU: {np.mean(scores)*100.0}")